AEMatter
====

**Revisiting Context Aggregation for Image Matting**

 * Paper: https://arxiv.org/pdf/2304.01171

![AEMatter Overview](../assets/aematter_overview.png)

```bash
git clone https://github.com/aipixel/AEMatter.git
```

## Download the model
 * AEMatter https://mega.nz/file/7N4AEKrS#L4h3Cm2qLMMbwBGm1lyGOmVDTXJwDMAi4BlBauqNHrI

 * Also, download swin model from huggingface

In [ ]:
import torch
from transformers import AutoModel, AutoImageProcessor

# Load model and processor
model_name = "microsoft/swin-tiny-patch4-window7-224"
swin_model = AutoModel.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name)

# Save model as .pth
torch.save(
    {"model": swin_model.state_dict()},
    "swin_tiny_patch4_window7_224.pth"
)
print("Model weights saved as swin_tiny_patch4_window7_224.pth")


Model weights saved as swin_tiny_patch4_window7_224.pth


In [6]:
import os
import sys

import cv2
from PIL import Image
import numpy as np
import torch

sys.path.append("AEMatter_repo")
import model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")